In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Load Dataset
file_path = "/content/Online Retail.xlsx"
df = pd.read_excel(file_path)

# Data Cleaning
df.dropna(subset=["CustomerID"], inplace=True)
df = df[df["Quantity"] > 0]
df["TotalPrice"] = df["Quantity"] * df["UnitPrice"]

# RFM Calculation
snapshot_date = df["InvoiceDate"].max() + pd.Timedelta(days=1)

rfm = df.groupby("CustomerID").agg({
    "InvoiceDate": lambda x: (snapshot_date - x.max()).days,  # Recency
    "InvoiceNo": "nunique",                                   # Frequency
    "TotalPrice": "sum"                                       # Monetary
}).reset_index()

rfm.rename(columns={
    "InvoiceDate": "Recency",
    "InvoiceNo": "Frequency",
    "TotalPrice": "Monetary"
}, inplace=True)

# Scoring
rfm["R_Score"] = pd.qcut(rfm["Recency"], 4, labels=[4, 3, 2, 1])
rfm["F_Score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 4, labels=[1, 2, 3, 4])
rfm["M_Score"] = pd.qcut(rfm["Monetary"], 4, labels=[1, 2, 3, 4])

rfm["RFM_Segment"] = rfm["R_Score"].astype(str) + rfm["F_Score"].astype(str) + rfm["M_Score"].astype(str)
rfm["RFM_Score"] = rfm[["R_Score", "F_Score", "M_Score"]].astype(int).sum(axis=1)

# Segment Naming
def segment_customer(row):
    if row["RFM_Score"] >= 9:
        return "Champions"
    elif row["RFM_Score"] >= 8:
        return "Loyal Customers"
    elif row["RFM_Score"] >= 6:
        return "Potential Loyalists"
    elif row["RFM_Score"] >= 4:
        return "At Risk"
    else:
        return "Hibernating"

rfm["Segment"] = rfm.apply(segment_customer, axis=1)

# Marketing Actions Column
marketing_actions = {
    "Champions": "Reward them with exclusive offers and early access.",
    "Loyal Customers": "Offer loyalty programs or discounts.",
    "Potential Loyalists": "Target with special deals to encourage repeat purchases.",
    "At Risk": "Send win-back campaigns or discounts.",
    "Hibernating": "Re-engage with targeted ads or email reminders."
}

rfm["Marketing_Idea"] = rfm["Segment"].map(marketing_actions)

# Display Top Customers Table
rfm_display = rfm[[
    "CustomerID", "Recency", "Frequency", "Monetary",
    "R_Score", "F_Score", "M_Score", "RFM_Score", "Segment", "Marketing_Idea"
]].sort_values("RFM_Score", ascending=False)

print("\nTop 10 Customers by RFM Score with Marketing Ideas:\n")
print(rfm_display.head(10))

# Save to Excel
output_path = "/content/RFM_Segments_With_Ideas.xlsx"
rfm_display.to_excel(output_path, index=False)
print(f"\nFull RFM table with marketing ideas saved to: {output_path}")

# Visualization
# Bar chart: Segment counts
plt.figure(figsize=(8,5))
sns.countplot(data=rfm, x="Segment", order=rfm["Segment"].value_counts().index, palette="viridis")
plt.title("Customer Segments")
plt.ylabel("Number of Customers")
plt.xticks(rotation=45)
plt.show()

# Heatmap: Average RFM values per segment
rfm_agg = rfm.groupby("Segment").agg({
    "Recency": "mean",
    "Frequency": "mean",
    "Monetary": "mean"
}).round(1)

plt.figure(figsize=(8,5))
sns.heatmap(rfm_agg, annot=True, fmt=".1f", cmap="Blues")
plt.title("Average RFM Values by Segment")
plt.show()



Top 10 Customers by RFM Score with Marketing Ideas:

      CustomerID  Recency  Frequency  Monetary R_Score F_Score M_Score  \
4337     18283.0        4         16   2094.88       4       4       4   
15       12362.0        3         10   5226.23       4       4       4   
2326     15514.0       12         11   4139.46       4       4       4   
1267     14062.0       10         12  10464.85       4       4       4   
1266     14060.0        5         19   6140.42       4       4       4   
4260     18180.0        5          6   1843.75       4       4       4   
4259     18179.0        9          6   1793.17       4       4       4   
61       12423.0        1          8   1859.31       4       4       4   
56       12417.0        3          9   3649.10       4       4       4   
4253     18172.0       15         20   7561.68       4       4       4   

      RFM_Score    Segment                                     Marketing_Idea  
4337         12  Champions  Reward them with exclus

/tmp/ipython-input-2349549759.py:81: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.countplot(data=rfm, x="Segment", order=rfm["Segment"].value_counts().index, palette="viridis")
